In [1]:
# import libraries
import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
# read the dataframe
data_path = 'Solcast_data_india.csv'
df = pd.read_csv(data_path)

In [3]:
df.sample(5)

,PeriodEnd,PeriodStart,Period,AirTemp,CloudOpacity,Dhi,Dni,Ghi,PrecipitableWater,RelativeHumidity,SurfacePressure,WindDirection10m,WindSpeed10m
21262,2021-06-04T00:00:00Z,2021-06-03T23:00:00Z,PT60M,31.9,58.4,0,0,0,35.6,32.9,980.3,84,2.4
15787,2020-10-18T21:00:00Z,2020-10-18T20:00:00Z,PT60M,27.0,53.9,0,0,0,10.7,17.3,981.9,320,1.7
6831,2019-10-11T17:00:00Z,2019-10-11T16:00:00Z,PT60M,28.9,0.0,0,0,0,22.9,37.5,985.4,21,1.9
10585,2020-03-16T03:00:00Z,2020-03-16T02:00:00Z,PT60M,18.1,69.3,88,0,88,15.2,56.3,991.2,318,1.7
25246,2021-11-17T00:00:00Z,2021-11-16T23:00:00Z,PT60M,19.3,20.0,0,0,0,9.2,36.5,988.6,67,0.9


### Changing the names and types of the columns

In [4]:
old_names = ['PeriodEnd', 'PeriodStart', 'AirTemp', 'Dhi', 'Dni', 'Ghi','PrecipitableWater', 'RelativeHumidity',
             'SurfacePressure', 'WindDirection10m', 'WindSpeed10m'] 
new_names = ['PeriodEnd','PeriodStart','Temperature', 'DHI', 'DNI', 'Radiation', 'Precipitation', 'Humidity', 
             'Pressure', 'WindDirection', 'WindSpeed']
df.rename(columns = dict(zip(old_names, new_names)), inplace = True)
df = df.drop(columns=['Period'])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26323 entries, 0 to 26322
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PeriodEnd      26323 non-null  object 
 1   PeriodStart    26323 non-null  object 
 2   Temperature    26323 non-null  float64
 3   CloudOpacity   26323 non-null  float64
 4   DHI            26323 non-null  int64  
 5   DNI            26323 non-null  int64  
 6   Radiation      26323 non-null  int64  
 7   Precipitation  26323 non-null  float64
 8   Humidity       26323 non-null  float64
 9   Pressure       26323 non-null  float64
 10  WindDirection  26323 non-null  int64  
 11  WindSpeed      26323 non-null  float64
dtypes: float64(6), int64(4), object(2)
memory usage: 2.4+ MB


In [6]:
 # interpret columns as appropriate data types to ensure compatibility
for col in ['Radiation', 'Temperature', 'Pressure', 'Humidity', 'WindDirection', 'WindSpeed']:
    df[col] = df[col].astype(float)

for col in ['PeriodStart', 'PeriodEnd']:
    df[col] = pd.to_datetime(df[col]).dt.to_period('T').dt.to_timestamp()

C:\Users\HP\anaconda3\lib\site-packages\pandas\core\arrays\datetimes.py:1101: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


In [7]:
df.head(5)

,PeriodEnd,PeriodStart,Temperature,CloudOpacity,DHI,DNI,Radiation,Precipitation,Humidity,Pressure,WindDirection,WindSpeed
0,2018-12-31 02:00:00,2018-12-31 01:00:00,7.4,0.0,2,5,2.0,9.9,89.1,997.1,298.0,2.7
1,2018-12-31 03:00:00,2018-12-31 02:00:00,8.1,25.8,68,11,69.0,9.9,86.1,997.6,294.0,3.2
2,2018-12-31 04:00:00,2018-12-31 03:00:00,10.1,10.1,143,320,257.0,10.0,77.7,997.8,293.0,3.4
3,2018-12-31 05:00:00,2018-12-31 04:00:00,13.4,0.0,144,660,451.0,10.0,65.7,997.8,293.0,3.2
4,2018-12-31 06:00:00,2018-12-31 05:00:00,16.8,0.0,169,716,572.0,10.1,55.9,997.7,294.0,3.1


### Aplying correct time zone and winter/summer time

In [8]:
df['PeriodStart'] = df['PeriodStart'].dt.tz_localize('UTC').dt.tz_convert('Europe/Paris')
df['PeriodEnd'] = df['PeriodEnd'].dt.tz_localize('UTC').dt.tz_convert('Europe/Paris')

### Create sunrise and sunset feature

In [9]:
# Create new features(year, month, day) from start and end time of data collection  
df['Year'] = pd.to_datetime(df['PeriodEnd']).dt.year
df['Month'] = pd.to_datetime(df['PeriodEnd']).dt.month

df['HourPS'] = pd.to_datetime(df['PeriodStart']).dt.hour
df['HourPE'] = pd.to_datetime(df['PeriodEnd']).dt.hour

In [10]:
df.loc[df['Radiation'] == 0]['HourPS'].unique()

array([13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,  0,  1,  2],
      dtype=int64)

In [11]:
df.loc[df['Radiation'] == 0]['HourPE'].unique()

array([14, 15, 16, 17, 18, 19, 20, 21, 22, 23,  0,  1,  2,  3],
      dtype=int64)

In [12]:
# create a new feature for sunrise using the function below; fill time for sunrise
def time_to_sunrise(row):
    """
    A function for identifying time of sunrise.
    """
    value = 0        
    if row['HourPS'] == 8 and row['HourPE'] == 9 and row['Radiation'] > 0:
        value= 9
    if row['HourPS'] == 7 and row['HourPE'] == 8 and row['Radiation'] > 0:
        value= 8
    elif row['HourPS'] == 6 and row['HourPE'] == 7 and row['Radiation'] > 0:
        value= 7
    elif row['HourPS'] == 5 and row['HourPE'] == 6 and row['Radiation'] > 0:
        value= 6
    elif row['HourPS'] == 4 and row['HourPE'] == 5 and row['Radiation'] > 0:
        value= 5
    elif row['HourPS'] == 3 and row['HourPE'] == 4 and row['Radiation'] > 0:
        value= 4
    return value

# create a new feature for sunset using the function below; fill time for sunset
def time_to_sunset(row):
    """
    A function for identifying time of sunset.
    """
    value= 0
    if row['HourPS'] == 15 and row['HourPE'] == 16 and row['Radiation']> 0:
        value= 16
    elif row['HourPS'] == 16 and row['HourPE'] == 17 and row['Radiation']> 0:
        value= 17
    elif row['HourPS'] == 17 and row['HourPE']== 18 and row['Radiation']> 0:
        value= 18
    elif row['HourPS'] == 18 and row['HourPE']== 19 and row['Radiation']> 0:
        value= 19
    elif row['HourPS'] == 19 and row['HourPE']== 20 and row['Radiation']> 0:
        value= 20
    elif row['HourPS'] == 20 and row['HourPE']== 21 and row['Radiation']> 0:
        value= 21
    elif row['HourPS'] == 21 and row['HourPE']== 22 and row['Radiation']> 0:
        value= 22
    elif row['HourPS'] == 22 and row['HourPE']== 23 and row['Radiation']> 0:
        value= 23
    return value

In [13]:
#Create a new column using the function above
df['Sunrise'] = df.apply(time_to_sunrise, axis=1)

#Create a new column using the function above
df['Sunset'] = df.apply(time_to_sunset, axis=1)

In [14]:
df.Sunrise.unique(), df.Sunset.unique()

(array([0, 4, 5, 6, 7, 8, 9], dtype=int64), array([ 0, 16], dtype=int64))

### Removing night time data

In [15]:
# changing integer values to datetime format
df = df.loc[~((df['Radiation'] == 0) & (df['DNI'] == 0) & (df['DHI'] == 0)),:]

### Grouping by day

In [16]:
df['date'] = pd.to_datetime(df['PeriodEnd']).dt.date

for col in ['Sunrise', 'Sunset']:
    df[col] = df[col].astype(str)
    
agg_dict = {
    'Temperature':'mean',
    'CloudOpacity':'mean',
    'DHI':'mean',
    'DNI':'mean',
    'Radiation':'mean',
    'Precipitation':'mean',
    'Humidity':'mean',
    'Pressure':'mean',
    'WindDirection':'mean',
    'WindSpeed':'mean',
    'Sunrise':'max',
    'Sunset':'max',
}

df_grouped = df.groupby(['date']).agg(agg_dict)
df_grouped.reset_index(inplace=True)
df_grouped

,date,Temperature,CloudOpacity,DHI,DNI,Radiation,Precipitation,Humidity,Pressure,WindDirection,WindSpeed,Sunrise,Sunset
0,2018-12-31,15.872727,3.263636,131.909091,418.636364,342.000000,10.272727,58.927273,996.481818,296.363636,3.336364,9,0
1,2019-01-01,15.818182,9.027273,169.545455,233.909091,292.545455,12.090909,64.390909,995.645455,267.454545,2.272727,9,0
2,2019-01-02,16.909091,8.045455,175.727273,203.454545,287.727273,14.318182,64.427273,995.681818,109.090909,1.863636,9,0
3,2019-01-03,16.554545,7.318182,164.454545,276.727273,307.090909,12.190909,64.500000,996.545455,299.272727,3.454545,9,0
4,2019-01-04,15.881818,44.790909,127.636364,73.636364,157.727273,14.763636,64.054545,995.372727,307.545455,2.190909,9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,2021-12-27,18.227273,11.336364,177.272727,168.090909,265.818182,16.645455,47.954545,993.581818,113.909091,1.263636,9,0
1093,2021-12-28,16.630000,82.190000,32.300000,0.000000,32.300000,20.330000,50.010000,992.910000,107.300000,2.040000,9,0
1094,2021-12-29,17.418182,8.336364,168.000000,234.090909,289.000000,11.936364,48.327273,993.118182,258.454545,1.990909,9,0
1095,2021-12-30,15.890909,20.800000,161.545455,184.454545,244.090909,5.981818,27.427273,997.872727,168.636364,1.409091,9,0


In [17]:
df_grouped['Sunrise'] = df_grouped['Sunrise'].astype(int) 
df_grouped['Sunset'] = df_grouped['Sunset'].astype(int) 
df_grouped['Month'] = pd.to_datetime(df_grouped['date']).dt.month

In [18]:
df_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1097 entries, 0 to 1096
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           1097 non-null   object 
 1   Temperature    1097 non-null   float64
 2   CloudOpacity   1097 non-null   float64
 3   DHI            1097 non-null   float64
 4   DNI            1097 non-null   float64
 5   Radiation      1097 non-null   float64
 6   Precipitation  1097 non-null   float64
 7   Humidity       1097 non-null   float64
 8   Pressure       1097 non-null   float64
 9   WindDirection  1097 non-null   float64
 10  WindSpeed      1097 non-null   float64
 11  Sunrise        1097 non-null   int32  
 12  Sunset         1097 non-null   int32  
 13  Month          1097 non-null   int64  
dtypes: float64(10), int32(2), int64(1), object(1)
memory usage: 111.5+ KB


In [19]:
df_grouped.shape

(1097, 14)

In [22]:
output_path = 'solcast_cleaned_india.csv'
df_grouped.to_csv(output_path)